In [ ]:
import os

import numpy as np
import pandas as pd
import psutil

import spatioloji_s as sj
from spatioloji_s.spatial import point as spoint
from spatioloji_s.spatial import polygon as spoly
from spatioloji_s.spatial.polygon.neighborhoods import harmonize_niches

In [2]:
os.getcwd()


'/carc/scratch/projects/amitra2016502/spatioloji_s/example'

In [3]:
mem = psutil.virtual_memory()
print(f"Total memory available: {mem.total / 1024**3:.2f} GB")
cpus = os.cpu_count()
print(f"Total cpu cores available: {cpus}")
print(f"current work directory is {os.getcwd()}")

Total memory available: 251.40 GB
Total cpu cores available: 64
current work directory is /carc/scratch/projects/amitra2016502/spatioloji_s/example


## Section 1: Setup & FOV-wise Subsetting

In [10]:
# Load your spatioloji object
sp = sj.data.spatioloji.from_pickle('my_data_objects/run_workflow_impute_spatioloji.pickle')


Loading spatioloji from: my_data_objects/run_workflow_impute_spatioloji.pickle
✓ Loaded: 25,368 cells × 960 genes


In [11]:
print(f"Cells : {sp.n_cells:,}")
print(f"Genes : {sp.n_genes:,}")
print(f"FOVs  : {sp.n_fovs}")
print(f"FOV IDs (first 5): {list(sp.fov_index[:5])}")
print(f"\nCell meta columns: {list(sp.cell_meta.columns[:8])}")

Cells : 25,368
Genes : 960
FOVs  : 20
FOV IDs (first 5): ['001', '002', '004', '005', '006']

Cell meta columns: ['fov', 'cell_ID', 'Area', 'AspectRatio', 'CenterX_local_px', 'CenterY_local_px', 'CenterX_global_px', 'CenterY_global_px']


In [12]:
# ── Subset a single FOV ──────────────────────────────────────────────────────
fov_id = sp.fov_index[0]               # pick first FOV as example
sp_fov = sp.subset_by_fovs([fov_id])

print(f"\nFOV {fov_id}: {sp_fov.n_cells} cells")

# ── Helper: iterate all FOVs ─────────────────────────────────────────────────
def iter_fovs(sp):
    """Yield (fov_id, sp_fov) for each FOV."""
    for fov_id in sp.fov_index:
        yield fov_id, sp.subset_by_fovs([fov_id])


Subsetting by FOVs: 1 FOVs requested

Subsetting by cells: 724 cells requested

Initializing spatioloji
[1/11] Master indices: 724 cells × 960 genes
[2/11] Expression matrix: sparse (0.7 MB)
[3/10] Cell metadata: 83 columns
[4/11] Gene metadata: 11 columns
[5/11] Spatial coordinates: 724 cells
[6/11] Polygons: 20136 vertices for 724 cells
[7/11] Images: 1 total, 0 loaded
[8/11] FOV system: 1 FOVs
[9/11] Index maps created
[10/11] Validation: ✓ All consistent
[11/11] Layers and Embeddings initialized (empty)

Spatioloji Object Summary:
  Cells:              724
  Genes:              960
  FOVs:               1
  Images:             1 (0 loaded)
  Original Expression:         sparse
  Has polygons:       True
  Memory (approx):    4.3 MB

✓ Created subset: 724 cells, 1 FOVs
  Layers subsetted (cell axis): ['normalized_counts', 'log_normalized', 'scaled', 'magic_imputed']
  Embeddings subsetted: ['X_pca', 'X_pca_variance', 'X_pca_variance_ratio', 'X_umap']

FOV 001: 724 cells


## Section 2: Polygon-based Analysis (FOV-wise)

In [13]:
# ── Collectors (all results stored here, NOT in sp_fov) ──────────────────────
morph_all        = []   # pd.DataFrame per FOV: shape metrics per cell
morph_class_all  = []   # pd.DataFrame per FOV: shape class per cell
contact_all      = []   # pd.DataFrame per FOV: per-pair contact lengths/fractions
free_frac_all    = []   # pd.DataFrame per FOV: free boundary fraction per cell
comp_dict        = {}   # {fov_id: comp_df} for cross-FOV niche harmonization
enrich_all       = {}   # {fov_id: z_score_df} neighborhood enrichment
hotspot_all      = []   # pd.DataFrame per FOV: Gi* results per cell

for fov_id, sp_fov in iter_fovs(sp):
    print(f"\n{'='*55}\nFOV: {fov_id}  ({sp_fov.n_cells} cells)\n{'='*55}")

    # ── Step 1: Cell morphology ───────────────────────────────────────────────
    # Computes: area, perimeter, circularity, elongation, solidity,
    #           compactness, convexity (store=False → no side effects on sp_fov)
    morph = spoly.compute_morphology(sp_fov, store=True)
    morph['fov'] = fov_id
    morph_all.append(morph)

    # Rule-based shape classification: 'round' / 'elongated' / 'irregular'
    morph_class = spoly.classify_morphology(
        sp_fov, method='threshold', store=True
    )  # → pd.Series indexed by cell_id
    morph_class = morph_class.rename('morph_class').to_frame()
    morph_class['fov'] = fov_id
    morph_class_all.append(morph_class)

    # ── Step 2: Build polygon contact graph ───────────────────────────────────
    # Adjacency = cells whose boundaries physically touch
    # Graph stays FOV-local (correct: FOVs are physically disconnected)
    graph = spoly.build_contact_graph(sp_fov)

    # ── Step 3: Boundary quantification ──────────────────────────────────────
    # contact_fraction → pd.DataFrame:
    #   cell_a, cell_b, contact_length_a, contact_length_b, fraction_a, fraction_b
    #   contact_length_a = portion of cell_a's OWN boundary inside cell_b
    #   (asymmetric for overlapping cells; uses unary_union to avoid
    #    double-counting at triple junctions when storing total)
    cf = spoly.contact_fraction(sp_fov, graph, store=True)
    cf['fov'] = fov_id
    contact_all.append(cf)

    # free_boundary_fraction → pd.Series:
    #   1.0 = fully isolated (exposed to ECM)
    #   0.0 = fully enclosed by neighbors
    #   Uses unary_union of all contact segments per cell before measuring
    #   → guaranteed no double-counting at triple junctions
    ff = spoly.free_boundary_fraction(sp_fov, graph, store=True)
    ff_df = ff.to_frame(name='free_boundary_fraction')
    ff_df['total_contact_fraction'] = 1.0 - ff   # exact complement
    ff_df['fov'] = fov_id
    free_frac_all.append(ff_df)

    # ── Step 4: Neighborhood composition ─────────────────────────────────────
    # For each cell: fraction of neighbors belonging to each cell type
    # store=False → we keep it in comp_dict for global harmonization
    comp = spoly.neighborhood_composition(
        sp_fov, graph,
        group_col='leiden',   # ← your cell type column
        mode='fraction',
        store=True
    )  # → pd.DataFrame: cells × cell_types
    comp_dict[fov_id] = comp

    # ── Step 5: Neighborhood enrichment ──────────────────────────────────────
    # Permutation test: which cell type pairs contact more/less than expected?
    # z_scores > 0 → enriched contact, z_scores < 0 → avoidance
    enrich = spoly.neighborhood_enrichment(
        sp_fov, graph,
        group_col='leiden',
        n_permutations=500,
        seed=42
    )
    enrich_all[fov_id] = {
        'z_scores' : enrich['z_scores'],   # cell_type × cell_type DataFrame
        'p_values' : enrich['p_values'],
        'observed' : enrich['observed'],
        'expected' : enrich['expected'],
    }

    # ── Step 6: Hotspot detection ─────────────────────────────────────────────
    # Getis-Ord Gi*: clusters of cells with significantly high total_counts
    # z_score > 0 + p_value < 0.05 → hot spot (high-expression cluster)
    # z_score < 0 + p_value < 0.05 → cold spot
    hotspots = spoly.hotspot_detection(
        sp_fov, graph,
        metric='total_counts',
        store=True
    )  # → pd.DataFrame: gi_star, z_score, p_value per cell
    hotspots['fov'] = fov_id
    hotspot_all.append(hotspots)


# ── Combine per-FOV results ───────────────────────────────────────────────────
morph_df       = pd.concat(morph_all)       # shape metrics, indexed by cell_id
morph_class_df = pd.concat(morph_class_all) # morph_class + fov
contact_df     = pd.concat(contact_all)     # per-pair contact lengths/fractions
free_df        = pd.concat(free_frac_all)   # free_boundary_fraction + total + fov
hotspot_df     = pd.concat(hotspot_all)     # Gi* results, indexed by cell_id



Subsetting by FOVs: 1 FOVs requested

Subsetting by cells: 724 cells requested

Initializing spatioloji
[1/11] Master indices: 724 cells × 960 genes
[2/11] Expression matrix: sparse (0.7 MB)
[3/10] Cell metadata: 83 columns
[4/11] Gene metadata: 11 columns
[5/11] Spatial coordinates: 724 cells
[6/11] Polygons: 20136 vertices for 724 cells
[7/11] Images: 1 total, 0 loaded
[8/11] FOV system: 1 FOVs
[9/11] Index maps created
[10/11] Validation: ✓ All consistent
[11/11] Layers and Embeddings initialized (empty)

Spatioloji Object Summary:
  Cells:              724
  Genes:              960
  FOVs:               1
  Images:             1 (0 loaded)
  Original Expression:         sparse
  Has polygons:       True
  Memory (approx):    4.3 MB

✓ Created subset: 724 cells, 1 FOVs
  Layers subsetted (cell axis): ['normalized_counts', 'log_normalized', 'scaled', 'magic_imputed']
  Embeddings subsetted: ['X_pca', 'X_pca_variance', 'X_pca_variance_ratio', 'X_umap']

FOV: 001  (724 cells)

[Morpho

In [9]:
morph_df

,area,perimeter,circularity,elongation,solidity,compactness,convexity,fov
cell,,,,,,,,
001_70,4633.5,252.709841,0.911748,1.362989,1.0,0.861441,1.0,001
001_168,8477.5,334.975360,0.949406,1.145833,1.0,0.944488,1.0,001
001_675,5545.0,279.176444,0.894034,1.091262,1.0,0.971813,1.0,001
001_1048,4726.0,246.155662,0.980131,1.163435,1.0,0.941758,1.0,001
001_992,6479.0,324.334582,0.773983,1.816552,1.0,0.713458,1.0,001
...,...,...,...,...,...,...,...,...
022_565,4226.0,246.346422,0.875078,1.256777,1.0,0.889929,1.0,022
022_1301,3726.5,238.611499,0.822486,1.180237,1.0,0.901785,1.0,022
022_3479,4407.5,252.756986,0.866954,1.518861,1.0,0.817169,1.0,022


In [10]:
morph_class_df.morph_class.value_counts()

morph_class
round        23994
elongated     1283
irregular       91
Name: count, dtype: int64

In [11]:
contact_df

,cell_a,cell_b,distance,contact_length_a,contact_length_b,fraction_a,fraction_b,fov
0,001_70,001_552,0.0,55.827973,55.493743,0.220917,0.178913,001
1,001_70,001_524,0.0,43.540831,45.248091,0.172296,0.270027,001
2,001_675,001_678,0.0,78.478396,75.581655,0.281107,0.311172,001
3,001_675,001_662,0.0,53.667567,53.809676,0.192235,0.263469,001
4,001_675,001_691,0.0,48.834783,49.614121,0.174924,0.319730,001
...,...,...,...,...,...,...,...,...
2640,022_2192,022_2199,0.0,24.521684,24.129289,0.092962,0.087246,022
2641,022_677,022_2841,0.0,70.424086,69.737110,0.219111,0.284146,022
2642,022_3487,022_3485,0.0,15.055133,15.068559,0.071830,0.069947,022
2643,022_2002,022_2009,0.0,4.087269,4.080441,0.025574,0.020207,022


In [13]:
free_df

,free_boundary_fraction,total_contact_fraction,fov
cell,,,
001_70,0.606787,0.393213,001
001_168,1.000000,0.000000,001
001_675,0.351733,0.648267,001
001_1048,1.000000,0.000000,001
001_992,0.530576,0.469424,001
...,...,...,...
022_565,0.361375,0.638625,022
022_1301,0.794241,0.205759,022
022_3479,0.859952,0.140048,022


In [12]:
hotspot_df

,gi_star,z_score,p_value,fov
cell,,,,
001_70,-0.083520,-0.083520,0.933438,001
001_168,-0.831077,-0.831077,0.405930,001
001_675,0.207541,0.207541,0.835588,001
001_1048,-0.627896,-0.627896,0.530072,001
001_992,-1.187729,-1.187729,0.234940,001
...,...,...,...,...
022_565,0.203807,0.203807,0.838505,022
022_1301,-0.461129,-0.461129,0.644706,022
022_3479,-1.514230,-1.514230,0.129968,022


In [14]:
comp_dict

{'001':                  0    1         2         3    4    5    6
 cell                                                      
 001_70    0.000000  0.0  0.000000  1.000000  0.0  0.0  0.0
 001_168   0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0
 001_675   0.000000  0.0  0.000000  1.000000  0.0  0.0  0.0
 001_1048  0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0
 001_992   0.000000  0.0  0.000000  1.000000  0.0  0.0  0.0
 ...            ...  ...       ...       ...  ...  ...  ...
 001_510   0.000000  0.0  0.000000  1.000000  0.0  0.0  0.0
 001_816   0.333333  0.0  0.333333  0.333333  0.0  0.0  0.0
 001_342   1.000000  0.0  0.000000  0.000000  0.0  0.0  0.0
 001_52    0.000000  0.0  0.000000  0.000000  0.0  1.0  0.0
 001_235   0.000000  0.0  0.000000  1.000000  0.0  0.0  0.0
 
 [724 rows x 7 columns],
 '002':            0         1    2    3         4    5         6
 cell                                                     
 002_700  0.0  0.333333  0.0  0.0  0.333333  0.0  0.333333
 0

In [14]:
# ── Step 7: Global niche harmonization ───────────────────────────────────────
# Problem: niche_identification() per FOV gives locally arbitrary labels
#   (niche_0 in FOV1 ≠ niche_0 in FOV2)
# Solution: cluster all FOV composition vectors together globally
#   → labels reflect actual tissue environments, not FOV-specific numbering
# Note: graph stays FOV-local (correct), only clustering is global

global_niches = harmonize_niches(
    comp_dict,      # {fov_id: comp_df} collected in loop above
    n_niches=5,
    method='kmeans',
    seed=42
)  # → pd.Series: cell_id → 'niche_0' ... 'niche_4', NaN for isolated cells

# Write global niche labels back to the full sp object
sp.cell_meta['global_niche'] = global_niches.reindex(sp.cell_index)



[Neighborhoods] Harmonizing niches across 20 FOVs...
  → 25368 total cells, 21854 with neighbors
  → 7 cell types in composition space
  → KMeans inertia: 64874.8

  ✓ Global niche summary (5 niches):
    niche_0: 5005 cells | top types: 1=0.64, 6=0.14, 0=0.12
      FOVs: {'001': np.int64(119), '002': np.int64(129), '004': np.int64(193), '005': np.int64(439), '006': np.int64(303), '007': np.int64(364), '008': np.int64(141), '009': np.int64(303), '010': np.int64(310), '011': np.int64(195), '012': np.int64(68), '013': np.int64(198), '014': np.int64(298), '015': np.int64(194), '016': np.int64(393), '017': np.int64(393), '018': np.int64(95), '019': np.int64(234), '021': np.int64(404), '022': np.int64(232)}
    niche_1: 2320 cells | top types: 3=0.94, 0=0.04, 1=0.01
      FOVs: {'001': np.int64(279), '002': np.int64(5), '004': np.int64(7), '006': np.int64(169), '007': np.int64(9), '008': np.int64(45), '009': np.int64(58), '010': np.int64(121), '011': np.int64(38), '012': np.int64(299), '01

In [15]:
# ── Quick summaries ───────────────────────────────────────────────────────────

# Significant hotspots across all FOVs
hot_cells  = hotspot_df[(hotspot_df['p_value'] < 0.05) & (hotspot_df['z_score'] > 0)]
cold_cells = hotspot_df[(hotspot_df['p_value'] < 0.05) & (hotspot_df['z_score'] < 0)]
print(f"\nHotspot cells : {len(hot_cells)}")
print(f"Cold spot cells: {len(cold_cells)}")

# Niche distribution across FOVs
print("\nGlobal niche distribution:")
print(sp.cell_meta['global_niche'].value_counts(dropna=False))

# Top enriched cell type contacts per FOV
print("\nTop enriched contact pair per FOV:")
for fov_id, res in enrich_all.items():
    z = res['z_scores']
    # Mask diagonal (self-contact) before finding max
    np.fill_diagonal(z.values, np.nan)
    top_pair = z.stack().idxmax()
    top_z    = z.stack().max()
    print(f"  FOV {fov_id}: {top_pair[0]} – {top_pair[1]}  (z={top_z:.2f})")

# Morphology overview
print("\nMorphology class distribution:")
print(morph_class_df['morph_class'].value_counts())

# Boundary exposure overview
print("\nBoundary exposure (free_boundary_fraction):")
print(free_df['free_boundary_fraction'].describe().round(3))





Hotspot cells : 2037
Cold spot cells: 199

Global niche distribution:
global_niche
niche_2    6177
niche_3    5092
niche_0    5005
NaN        3514
niche_4    3260
niche_1    2320
Name: count, dtype: int64

Top enriched contact pair per FOV:
  FOV 001: 4 – 5  (z=1.81)
  FOV 002: 0 – 3  (z=4.37)
  FOV 004: 1 – 4  (z=4.39)
  FOV 005: 2 – 3  (z=2.28)
  FOV 006: 0 – 5  (z=1.49)
  FOV 007: 4 – 5  (z=2.32)
  FOV 008: 2 – 5  (z=4.93)
  FOV 009: 5 – 6  (z=1.54)
  FOV 010: 1 – 4  (z=3.05)
  FOV 011: 2 – 5  (z=3.22)
  FOV 012: 4 – 6  (z=1.66)
  FOV 013: 0 – 3  (z=4.29)
  FOV 014: 2 – 5  (z=4.34)
  FOV 015: 5 – 6  (z=2.59)
  FOV 016: 2 – 5  (z=8.65)
  FOV 017: 2 – 3  (z=3.43)
  FOV 018: 5 – 6  (z=3.22)
  FOV 019: 4 – 5  (z=7.95)
  FOV 021: 2 – 5  (z=6.86)
  FOV 022: 0 – 1  (z=5.03)

Morphology class distribution:
morph_class
round        23994
elongated     1283
irregular       91
Name: count, dtype: int64

Boundary exposure (free_boundary_fraction):
count    25368.000
mean         0.671
std     

## Section 3: Point-based Analysis (FOV-wise)

In [7]:
# ── User settings — update these to match your data ──────────────────────────

LAYER         = 'log_normalized'    # layer to use for all gene analysis
                              # must exist in sp.layers (sp.add_layer(...))
CELL_TYPE_COL = 'leiden'     # cell type column in cell_meta
K_NEIGHBORS   = 10           # KNN graph connectivity

# Genes to run full permutation Moran's I + LISA + Gi* on
GENES_OF_INTEREST = ['EPCAM', 'DCN']   # ← replace with your genes

# Cell type pairs for cross-type distance (source → target)
CROSS_TYPE_PAIRS = [
    ('0', '1'),
    ('1', '2'),
]

# ── Verify layer exists before starting ──────────────────────────────────────
if LAYER not in sp.layers:
    raise ValueError(
        f"Layer '{LAYER}' not found. "
        f"Available: {sp.list_layers()}. "
        f"Add it with: sp.add_layer('{LAYER}', your_matrix)"
    )
print(f"Using layer '{LAYER}' for gene-level spatial analysis.")


Using layer 'log_normalized' for gene-level spatial analysis.


In [8]:
# ── Collectors ────────────────────────────────────────────────────────────────
svg_all          = []   # SVG results per FOV
morans_all       = []   # global Moran's I for genes of interest
lisa_all         = {}   # {fov_id: {gene: LISA DataFrame}}
gi_all           = {}   # {fov_id: {gene: Gi* DataFrame}}
enrich_point_all = {}   # centroid-based neighborhood enrichment
ripley_all       = {}   # {fov_id: {cell_type: envelope}}
cross_dist_all   = []   # cross-type distances
nnd_all          = []   # nearest neighbor distances


for fov_id, sp_fov in iter_fovs(sp):
    print(f"\n{'='*55}\nFOV: {fov_id}  ({sp_fov.n_cells} cells)\n{'='*55}")

    # ── Step 1: KNN graph ─────────────────────────────────────────────────────
    # Centroid-based, stays FOV-local
    graph = spoint.build_knn_graph(sp_fov, k=K_NEIGHBORS, coord_type='global')

    # ── Step 2: SVG screening (all genes, fast analytical Moran's I) ─────────
    # Runs on normalized layer → fair comparison across genes
    # Use for discovery; validate top hits with full permutation in Step 3
    svg = spoint.spatially_variable_genes(
        sp_fov, graph,
        layer=LAYER,
        n_top=50,
        fdr_threshold=0.05
    )  # → pd.DataFrame: gene × [I, expected, variance, zscore, pvalue, fdr]
    svg['fov'] = fov_id
    svg_all.append(svg)

    # ── Step 3: Global Moran's I (full permutation, genes of interest) ────────
    # More robust than analytical Step 2 — use for genes you want to report
    # I > 0 → spatially clustered, I < 0 → dispersed, I ≈ 0 → random
    morans_rows = []
    for gene in GENES_OF_INTEREST:
        if gene not in sp_fov.gene_index:
            continue
        result = spoint.morans_i(
            sp_fov, graph,
            values=gene,
            layer=LAYER,
            local=False,
            n_permutations=999,   # use 199 during development for speed
            seed=42
        )  # → dict: I, expected, zscore, pvalue
        morans_rows.append({'gene': gene, 'fov': fov_id, **result})
    if morans_rows:
        morans_all.append(pd.DataFrame(morans_rows))

    # ── Step 4: Local Moran's I / LISA ───────────────────────────────────────
    # WHERE does each gene cluster spatially?
    # HH = high-expression cluster, LL = low-expression cluster
    # HL/LH = spatial outliers, ns = not significant
    fov_lisa = {}
    for gene in GENES_OF_INTEREST:
        if gene not in sp_fov.gene_index:
            continue
        lisa = spoint.morans_i(
            sp_fov, graph,
            values=gene,
            layer=LAYER,
            local=True,           # ← LISA mode
            n_permutations=199,
            seed=42
        )  # → pd.DataFrame: local_i, zscore, pvalue, spot_type (indexed by cell_id)
        lisa['fov'] = fov_id
        fov_lisa[gene] = lisa
    lisa_all[fov_id] = fov_lisa

    # ── Step 5: Getis-Ord Gi* hotspots per gene ───────────────────────────────
    # Gi* > 0, p < 0.05 → hotspot (high-expression spatial cluster)
    # Gi* < 0, p < 0.05 → coldspot
    # Unlike Moran's I, Gi* tells you the DIRECTION of clustering
    fov_gi = {}
    for gene in GENES_OF_INTEREST:
        if gene not in sp_fov.gene_index:
            continue
        gi = spoint.getis_ord_gi(
            sp_fov, graph,
            values=gene,
            layer=LAYER,
            star=True,
            permutation_pvalue=False,  # analytical (fast); True for publication
            store=False
        )  # → pd.DataFrame: gi, zscore, pvalue, spot_type (indexed by cell_id)
        gi['fov'] = fov_id
        fov_gi[gene] = gi
    gi_all[fov_id] = fov_gi

    # ── Step 6: Centroid-based neighborhood enrichment ────────────────────────
    # Compare with polygon enrichment (Section 2, Step 5):
    # differences reveal contact-topology vs. pure spatial proximity effects
    enrich = spoint.neighborhood_enrichment(
        sp_fov, graph,
        cell_type_col=CELL_TYPE_COL,
        n_permutations=500,
        seed=42
    )
    enrich_point_all[fov_id] = {
        'z_scores' : enrich['zscore'],
        'p_values' : enrich['pvalue'],
        'observed' : enrich['observed'],
        'expected' : enrich['expected'],
    }

    # ── Step 7: Ripley's L per cell type ─────────────────────────────────────
    # L(r) > 0 → cells cluster at radius r (relative to CSR)
    # L(r) < 0 → cells disperse at radius r
    # L(r) = 0 → complete spatial randomness (CSR)
    fov_ripley = {}
    for cell_type in sp_fov.cell_meta[CELL_TYPE_COL].dropna().unique():
        cell_ids = sp_fov.cell_meta.index[
            sp_fov.cell_meta[CELL_TYPE_COL] == cell_type
        ].tolist()

        if len(cell_ids) < 10:   # too few cells for reliable estimate
            continue

        envelope = spoint.simulation_envelope(
            sp_fov,
            function='L',
            cell_ids=cell_ids,
            n_simulations=99,    # increase to 999 for publication
            coord_type='global',
            seed=42
        )
        fov_ripley[cell_type] = envelope
    ripley_all[fov_id] = fov_ripley

    # ── Step 8: Cross-type distances ─────────────────────────────────────────
    # For each source cell: distance to its nearest target-type cell
    # Asymmetric: dist(A→B) ≠ dist(B→A)
    for source_type, target_type in CROSS_TYPE_PAIRS:
        types_present = sp_fov.cell_meta[CELL_TYPE_COL].dropna().unique()
        if source_type not in types_present or target_type not in types_present:
            continue

        dist_df = spoint.cross_type_distances(
            sp_fov,
            cell_type_col=CELL_TYPE_COL,
            source_type=source_type,
            target_type=target_type,
            coord_type='global',
            k=1,
            store=False
        )  # → pd.DataFrame: distance, target_cell_id (indexed by source cell_id)
        dist_df['source_type'] = source_type
        dist_df['target_type'] = target_type
        dist_df['fov'] = fov_id
        cross_dist_all.append(dist_df)

    # ── Step 9: Nearest neighbor distances ───────────────────────────────────
    # How close is each cell to its nearest neighbor?
    # Compare across FOVs to detect density differences between tissue regions
    nnd = spoint.nearest_neighbor_distances(
        sp_fov,
        coord_type='global',
        cell_type_col=CELL_TYPE_COL,
        k=1,
        store=False
    )  # → pd.DataFrame: nnd, nn_cell_id (indexed by cell_id)
    nnd['fov'] = fov_id
    nnd_all.append(nnd)


# ── Combine per-FOV results ───────────────────────────────────────────────────
svg_df        = pd.concat(svg_all)
nnd_df        = pd.concat(nnd_all)
cross_dist_df = pd.concat(cross_dist_all) if cross_dist_all else pd.DataFrame()
morans_df     = pd.concat(morans_all)     if morans_all     else pd.DataFrame()

# lisa_all and gi_all kept as nested dicts {fov_id: {gene: DataFrame}}
# Access example: lisa_all['001']['GeneA']





Subsetting by FOVs: 1 FOVs requested

Subsetting by cells: 724 cells requested

Initializing spatioloji
[1/11] Master indices: 724 cells × 960 genes
[2/11] Expression matrix: sparse (0.7 MB)
[3/10] Cell metadata: 83 columns
[4/11] Gene metadata: 11 columns
[5/11] Spatial coordinates: 724 cells
[6/11] Polygons: 20136 vertices for 724 cells
[7/11] Images: 1 total, 0 loaded
[8/11] FOV system: 1 FOVs
[9/11] Index maps created
[10/11] Validation: ✓ All consistent
[11/11] Layers and Embeddings initialized (empty)

Spatioloji Object Summary:
  Cells:              724
  Genes:              960
  FOVs:               1
  Images:             1 (0 loaded)
  Original Expression:         sparse
  Has polygons:       True
  Memory (approx):    4.3 MB

✓ Created subset: 724 cells, 1 FOVs
  Layers subsetted (cell axis): ['normalized_counts', 'log_normalized', 'scaled', 'magic_imputed']
  Embeddings subsetted: ['X_pca', 'X_pca_variance', 'X_pca_variance_ratio', 'X_umap']

FOV: 001  (724 cells)
  ✓ KNN 

In [9]:
# ── Step 10: Proximity score (global, all FOVs combined) ─────────────────────
# Intentionally run on full sp — global summary of spatial co-location
# normalized < 1 → closer than random, > 1 → farther than random
proximity = spoint.proximity_score(
    sp,
    cell_type_col=CELL_TYPE_COL,
    coord_type='global',
    normalize=True
)
# proximity['median_distance'] → cell_type × cell_type DataFrame
# proximity['normalized']      → ratio to random expectation


# ── Quick summaries ───────────────────────────────────────────────────────────

# Top SVGs consistent across FOVs
sig_svg = svg_df[svg_df['fdr'] < 0.05]
if not sig_svg.empty:
    top_svg = (
        sig_svg.groupby(sig_svg.index)['I']
        .agg(mean_I='mean', n_fovs='count')
        .sort_values('mean_I', ascending=False)
        .head(20)
    )
    print(f"\nTop spatially variable genes (layer='{LAYER}'):")
    print(top_svg)

# Moran's I for genes of interest
if not morans_df.empty:
    print(f"\nMoran's I for genes of interest (layer='{LAYER}'):")
    print(morans_df.groupby('gene')[['I', 'pvalue']].mean().round(4))

# LISA: HH spot count per gene per FOV
print("\nLISA HH spot counts per gene per FOV:")
for fov_id, fov_lisa in lisa_all.items():
    for gene, lisa_df in fov_lisa.items():
        n_hh = (lisa_df['spot_type'] == 'HH').sum()
        print(f"  FOV {fov_id} | {gene}: {n_hh} HH cells")

# Gi* hotspot summary
print("\nGi* hotspot / coldspot counts per gene per FOV:")
for fov_id, fov_gi in gi_all.items():
    for gene, gi_df in fov_gi.items():
        n_hot  = (gi_df['spot_type'] == 'hotspot').sum()
        n_cold = (gi_df['spot_type'] == 'coldspot').sum()
        print(f"  FOV {fov_id} | {gene}: {n_hot} hotspots, {n_cold} coldspots")

# Cross-type distance summary
if not cross_dist_df.empty:
    print("\nMedian cross-type distance per FOV:")
    print(
        cross_dist_df
        .groupby(['source_type', 'target_type', 'fov'])['distance']
        .median()
        .unstack('fov')
        .round(1)
    )

# NND overview
print("\nMedian nearest-neighbor distance per FOV:")
print(nnd_df.groupby('fov')['nnd'].median().round(2))

# Global proximity
print("\nGlobal proximity score (normalized, < 1 = closer than random):")
print(proximity['normalized'].round(2))

  Computing proximity for 7 types...
  ✓ Proximity scores: 7×7 matrix
    Closest pair: 6→1 (norm=0.88)

Top spatially variable genes (layer='log_normalized'):
          mean_I  n_fovs
CD24    0.332765      10
PIGR    0.315436       2
EPCAM   0.270501      10
SST     0.269379       2
SPP1    0.268913      20
PTGS1   0.256164       7
LYZ     0.238659      20
GPNMB   0.234079      20
B2M     0.209321      20
CLU     0.200981       9
CD68    0.197787      20
IGKC    0.186352      18
ESR1    0.184175       8
PSAP    0.181980      20
SLPI    0.180399      13
DDR1    0.179781       9
INHBB   0.179155       4
MECOM   0.178780       7
JCHAIN  0.172857      19
IGHG2   0.171151      17

Moran's I for genes of interest (layer='log_normalized'):
            I  pvalue
gene                 
DCN    0.0456  0.1158
EPCAM  0.1354  0.2198

LISA HH spot counts per gene per FOV:
  FOV 001 | EPCAM: 0 HH cells
  FOV 001 | DCN: 11 HH cells
  FOV 002 | EPCAM: 1 HH cells
  FOV 002 | DCN: 2 HH cells
  FOV 004 | 

In [16]:
import datetime
import pickle
from pathlib import Path

# ── Output path — change as needed ───────────────────────────────────────────
OUTPUT_PATH = "my_data_objects/spatial_analysis_results.pkl"


# ── Package all results ───────────────────────────────────────────────────────
results = {

    # ── Metadata ──────────────────────────────────────────────────────────────
    'meta': {
        'saved_at'     : datetime.datetime.now().isoformat(),
        'n_cells'      : sp.n_cells,
        'n_fovs'       : sp.n_fovs,
        'n_genes'      : sp.n_genes,
        'cell_type_col': CELL_TYPE_COL,
        'layer_used'   : LAYER,
    },

    # ── Section 2: Polygon-based results ─────────────────────────────────────

    # Step 1 — Morphology
    # pd.DataFrame: cell_id × [area, perimeter, circularity, elongation,
    #               solidity, compactness, convexity, fov]
    'morph_df'       : morph_df,

    # pd.DataFrame: cell_id × [morph_class, fov]
    # morph_class values: 'round' / 'elongated' / 'irregular'
    'morph_class_df' : morph_class_df,

    # Step 3 — Boundary quantification
    # pd.DataFrame: [cell_a, cell_b, contact_length_a, contact_length_b,
    #               fraction_a, fraction_b, fov]
    'contact_df'     : contact_df,

    # pd.DataFrame: cell_id × [free_boundary_fraction,
    #               total_contact_fraction, fov]
    'free_df'        : free_df,

    # Step 4 — Neighborhood composition (pre-harmonization, per-FOV)
    # dict: {fov_id → pd.DataFrame (cells × cell_types, fraction of each type)}
    'comp_dict'      : comp_dict,

    # Step 5 — Neighborhood enrichment (polygon/contact-based)
    # dict: {fov_id → {'z_scores', 'p_values', 'observed', 'expected'}}
    #   z_scores: cell_type × cell_type DataFrame
    'enrich_poly'    : enrich_all,

    # Step 6 — Hotspot detection (Gi* on total_counts)
    # pd.DataFrame: cell_id × [gi_star, z_score, p_value, fov]
    'hotspot_df'     : hotspot_df,

    # Step 7 — Global niches (cross-FOV harmonized labels)
    # pd.Series: cell_id → 'niche_0' ... 'niche_N'
    # (also written to sp.cell_meta['global_niche'])
    'global_niches'  : global_niches,

    # ── Section 3: Point-based results ───────────────────────────────────────

    # Step 2 — Spatially variable genes (analytical Moran's I, all genes)
    # pd.DataFrame: gene × [I, expected, variance, zscore, pvalue, fdr, fov]
    'svg_df'         : svg_df,

    # Step 3 — Global Moran's I (full permutation, genes of interest)
    # pd.DataFrame: [gene, fov, I, expected, zscore, pvalue]
    'morans_df'      : morans_df,

    # Step 4 — Local Moran's I / LISA
    # dict: {fov_id → {gene → pd.DataFrame(local_i, zscore, pvalue, spot_type, fov)}}
    # spot_type: 'HH', 'LL', 'HL', 'LH', 'ns'
    'lisa_all'       : lisa_all,

    # Step 5 — Getis-Ord Gi* per gene
    # dict: {fov_id → {gene → pd.DataFrame(gi, zscore, pvalue, spot_type, fov)}}
    # spot_type: 'hotspot', 'coldspot', 'ns'
    'gi_all'         : gi_all,

    # Step 6 — Centroid-based neighborhood enrichment
    # dict: {fov_id → {'z_scores', 'p_values', 'observed', 'expected'}}
    'enrich_point'   : enrich_point_all,

    # Step 7 — Ripley's L simulation envelopes
    # dict: {fov_id → {cell_type → envelope_dict}}
    'ripley_all'     : ripley_all,

    # Step 8 — Cross-type nearest distances
    # pd.DataFrame: [distance, target_cell_id, source_type, target_type, fov]
    'cross_dist_df'  : cross_dist_df,

    # Step 9 — Nearest neighbor distances (cell packing per FOV)
    # pd.DataFrame: [nnd, nn_cell_id, fov] (indexed by cell_id)
    'nnd_df'         : nnd_df,

    # Step 10 — Global proximity score
    # dict: {'median_distance': DataFrame, 'normalized': DataFrame}
    'proximity'      : proximity,
}


# ── Save ──────────────────────────────────────────────────────────────────────
output_path = Path(OUTPUT_PATH)
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'wb') as f:
    pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)

size_mb = output_path.stat().st_size / (1024 ** 2)
print(f"✓ Saved spatial analysis results → {output_path}  ({size_mb:.1f} MB)")
print(f"  Keys: {list(results.keys())}")

✓ Saved spatial analysis results → my_data_objects/spatial_analysis_results.pkl  (12.1 MB)
  Keys: ['meta', 'morph_df', 'morph_class_df', 'contact_df', 'free_df', 'comp_dict', 'enrich_poly', 'hotspot_df', 'global_niches', 'svg_df', 'morans_df', 'lisa_all', 'gi_all', 'enrich_point', 'ripley_all', 'cross_dist_df', 'nnd_df', 'proximity']
